## Create Points for StreamStats

__Description__:

__Input__:

__Output__:


*Authors*: sputnam@Dewberry.com & slawler@Dewberry.com

#### Load libraries and Python options:

In [1]:
import os
import collections 
import pandas as pd
import numpy as np
import geopandas as gpd
from osgeo import gdal, ogr,osr
from shapely.geometry import Point
from StreamStats_Points import*

### Load the masked stream grid:

##### Specify:

In [2]:
path=r'C:\Users\sputnam\Documents\GitHub\usgs-tools\results\rock_creek_clip.tif' #Load the stream grid raster which was masked by the catchment polygon

##### Load:

In [3]:
sg = StreamGrid(path) #Open the stream grid raster and create an object

crs=sg.crs_value() #Extract the coordinate reference system value (epsg) for the raster
print("epsg:",crs) #Print the value

df = sg.dataframe() #Create a dataframe from the stream grid data
df.replace(255, 0, inplace=True) #Replace 255 with 0, where 255 corresponds to the non-stream cells
df.head(n=2) 

epsg: 5070


,0,1,2,3,4,5,6,7,8,9,...,1205,1206,1207,1208,1209,1210,1211,1212,1213,1214
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Specify the pour point to set the start location of the search:

##### Specify:

In [4]:
lat=1925315.186 #latitude of the pourpoint at the catchment outlet
lon=1616784.964 #longitude of the pourpoint

##### Convert the lat/lon to row/column in the stream grid dataframe and extract the cell size:

In [5]:
pix_x, pix_y =coord2index(sg, lat, lon) #Transform the lat and lon values to the row/column location with the stream grid dataframe
pourpoint=[(pix_x, pix_y)] #Add these values to a list as a touple
print("Pourpoint XY:", pourpoint)

cellsize=sg.cell_size() #Raster cell size in meters
print("The Cell Size:", cellsize)

Pourpoint XY: [(877, 1848)]
The Cell Size: 10.0


### Move up the stream and identify the confluences:

##### Specify parameters and intalize objects:

In [6]:
nogo=[] #Empty list to store the stream cells that we do not want to return to since we have already searched them
confluence_pairs=[] #Empty list to store the identified confluence pairs
save_confluence=[] #Empty list to store the location of confluences that are three cells away from the original confluence location
cnum=0
count=0

starting_point=pourpoint[0]+(cnum,) #The starting point of the stream network where we want to start searching for confluences

nogo.append(pourpoint[0]) #Add the starting point to the no go list

##### Identify confluences:

In [7]:
while len(starting_point)>0:
    count+=1
    cnum=count
    
    next_cell, nogo=MoveUpstream(df, starting_point, nogo, cnum)  
    
    if len(next_cell) == 1:
        nogo=nogo+[next_cell[0][:2]]
        starting_point = next_cell[0]
        
    else:
        if len(next_cell)>1:
            next_cellwocnum=remove_cnum(next_cell)
            nogo=nogo+next_cellwocnum
            confluence_pairs=confluence_pairs+next_cell
        if len(confluence_pairs)>0:
            starting_point=confluence_pairs[0]
            confluence_pairs.remove(starting_point)
            cnum=starting_point[2]
            
            i=0
            while i<2:
                next_cell, nogo=MoveUpstream(df, starting_point, nogo, cnum)
                
                if len(next_cell) == 1:
                    nogo=nogo+[next_cell[0][:2]]
                    starting_point = next_cell[0]
                    i+=1
                    continue
                elif len(next_cell)>1:
                    confluence_pairs=confluence_pairs+next_cell
                    i=2
                else:
                    i=2
                
            if len(next_cell) == 1:
                save_confluence.append(starting_point)
        else:
            starting_point=[]

### Remove superflous confluences:

##### Remove points where two stream cells were adjacent:

In [23]:
true_confluence=[]
confl_num=[]

for cell in save_confluence:
    confl_num.append(cell[2])

for cell in save_confluence:
    if confl_num.count(cell[2])>=2:
        true_confluence.append(cell)
        
false_confluence=list(set(save_confluence)-set(true_confluence))

print("All Points:", len(save_confluence), "True Confluences:", len(true_confluence))    

All Points: 614 True Confluences: 475


### Calculate the distance:

In [27]:
nogo=[]
dis_confluence=[]

walk_confluence=true_confluence.copy()
starting_point=walk_confluence[0]

walk_conflwocnum=[starting_point[:2]]
nogo=nogo+walk_conflwocnum

false_confluence=remove_cnum(false_confluence) #Remove the confluence numbers from the false confluence list

In [ ]:
while len(walk_confluence)>0:
        next_cell, nogo=MoveUpstream(df, starting_point, nogo)
        
        if len(next_cell)==1:
            step_dis=TrueDistance(starting_point[0], next_cell[0], cellsize)
            total_dis=step_dis+total_dis
            nogo=nogo+[next_cell[0][:2]]
            starting_point = next_cell[0]
            continue
            
        elif len(next_cell)==0:
            dis_confluence(walk_confluence[0]+(total_dis,)+('T',))
            walk_confluence.remove(walk_confluence[0])
            starting_point=walk_confluence[0]
            continue
            
        elif len(next_cell)>1:
            next_cellwocnum=remove_cnum(next_cell)
            if any(x in next_cellwocnum for x in false_confluence):
                for cell in next_cellwocnum:
                    if cell in false_confluence:
                        next_cell=[cell]
                step_dis=TrueDistance(starting_point[0], next_cell[0], cellsize)
                total_dis=step_dis+total_dis
                nogo=nogo+[next_cell[0][:2]]
                
                ####come back to this part of the code
            dis_confluence(walk_confluence[0]+(total_dis,)+('M',))
            walk_confluence.remove(walk_confluence[0])
            starting_point=walk_confluence[0]
            continue

In [29]:
any(x in next_cellwocnum for x in false_confluence)

In [36]:
b=[false_confluence[2],(44,33)]
b==false_confluence

False

True

### Save the results:

In [ ]:
lists=[save_confluence, true_confluence]
names=['save_confluence', 'true_confluence']

for i in range(len(lists)):
    longitude, latitude=index2coord(sg, lists[i])  #Transform the row/column value from the stream grid dataframe to latitude/longitude for each confluence
    gdf=geodataframe(longitude, latitude, crs) #Store the longitude/latitude for each confluence in a geodataframe
    gdf.to_file(filename = r'C:\Users\sputnam\Documents\GitHub\usgs-tools\results\{}.shp'.format(names[i])) #Export the geodataframe as a shapefule

# End